# Case study 2: spam email detection from ACIC (logistic)
This notebook includes experiments from Case Study 2 from the paper Multi-Source Causal Inference Using Control Variates. Specifically, this notebook contains experiments using the logistic model with interaction between $X$ and $Z$ to estimate the ATE and odds ratios.

This experiment uses $n_2 = 10,000$ samples for the dataset without selection bias.

In [1]:
import numpy as np
import pandas as pd

import data_sampler
import bootstrap

In [2]:
df_orig = pd.read_csv('spam_binMod11.csv', sep=",")

In [3]:
df_orig = df_orig[:3000]

In [5]:
df_orig.describe()

,Y,A,V1,V2,V3,V4,V5,V6,V7,V8,...,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,...,3000.00000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,0.294667,0.225000,0.319917,0.098823,0.118190,0.10566,0.083893,0.246810,0.062010,0.529727,...,0.15698,0.186447,1.699940,0.101100,0.824350,0.116147,0.107973,0.088857,0.528137,0.972863
std,0.455969,0.417652,0.723370,0.286872,0.426914,0.37970,0.264619,0.591946,0.206386,0.828893,...,0.48656,0.506001,1.778479,0.720557,1.189208,1.044481,0.345529,0.349962,1.744793,0.762200
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.479799
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.090000,...,0.00000,0.000000,1.380000,0.000000,0.270000,0.000000,0.000000,0.000000,0.000000,0.832909
75%,1.000000,0.000000,0.380000,0.000000,0.000000,0.00000,0.000000,0.200000,0.000000,0.770000,...,0.00000,0.000000,2.632500,0.000000,1.290000,0.000000,0.000000,0.000000,0.000000,1.332102
max,1.000000,1.000000,10.000000,5.880000,7.270000,6.06000,3.330000,11.110000,2.610000,6.450000,...,5.12000,9.090000,14.280000,18.180000,10.710000,17.100000,3.620000,9.090000,20.830000,7.005336


In [6]:
Y_COLUMN = 'Y'
Z_COLUMN = 'A'
X_COLUMNS = ['V%d' % i for i in range(1,23)]

# Load selection biased data

In [8]:
df_large = pd.read_csv('spam_binMod1_large1.csv', sep=",")

In [9]:
data_sampler_selection_bias = data_sampler.DataSampler(Z_COLUMN, X_COLUMNS, Y_COLUMN)
df_selection_bias = data_sampler_selection_bias.selection_bias_filter(df_large)
df_selection_bias = df_selection_bias.sample(10*len(df_orig), replace=False) 

In [10]:
df_selection_bias.describe()

,Y,A,V1,V2,V3,V4,V5,V6,V7,V8,...,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,0.802700,0.183833,0.183193,0.050820,0.035835,0.055444,0.048844,0.195476,0.030138,0.603977,...,0.078789,0.115841,1.191607,0.027355,0.457500,0.042528,0.033293,0.031051,1.339257,0.743643
std,0.397967,0.387355,0.516133,0.182718,0.222851,0.281944,0.197846,0.587197,0.150631,0.999442,...,0.289406,0.418777,1.615539,0.258058,0.919572,0.603591,0.194806,0.218721,2.478006,0.561927
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.393393
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.686626
75%,1.000000,0.000000,0.090000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.860000,...,0.000000,0.000000,1.900000,0.000000,0.630000,0.000000,0.000000,0.000000,1.840000,1.002468
max,1.000000,1.000000,10.000000,5.880000,7.270000,11.110000,5.260000,11.110000,2.610000,9.670000,...,5.060000,9.090000,18.750000,18.180000,11.110000,17.100000,5.450000,9.750000,20.830000,7.005336


# Logistic regression model with interaction terms

In this section, we assume that the data generating outcome model is

$$P(Y=1 | Z = z, X = x) = \frac{e^{\beta_0 + \beta_1 z + \beta_2 ^T x + \beta_3 ^T xz}}{1 + e^{\beta_0 + \beta_1 z + \beta_2^T x + \beta_3 ^T xz}}$$

This allows for linear heterogenous effects in $x$.

# Compute ATE estimates with and without control variate

In [30]:
def ATE_estimator_fn_interaction(df_input):
    data_sampler_interaction_logistic = data_sampler.DataSamplerInteractionLogistic(Z_COLUMN, X_COLUMNS, Y_COLUMN)
    data_sampler_interaction_logistic.fit_outcome(df_input)
    return data_sampler_interaction_logistic.get_ATE_estimate(df_input)

def CV_estimator_fn_interaction(df_input_obs, df_input_bias):
    data_sampler_interaction_logistic = data_sampler.DataSamplerInteractionLogistic(Z_COLUMN, X_COLUMNS, Y_COLUMN)
    OR_xs = df_input_obs[X_COLUMNS] # Average over all xs in the observational dataset.
    # Estimate OR from observational dataset
    data_sampler_interaction_logistic.fit_outcome(df_input_obs)
    OR_obs = np.mean(data_sampler_interaction_logistic.get_conditional_OR_estimates(OR_xs))
    # Estimate OR from selection bias dataset
    data_sampler_interaction_logistic.fit_outcome(df_input_bias)
    OR_bias = np.mean(data_sampler_interaction_logistic.get_conditional_OR_estimates(OR_xs))
    return OR_obs - OR_bias

CV_samples, ATE_hat_samples, _ = bootstrap.run_bootstrap_df(df_obs=df_orig, 
              df_bias=df_selection_bias, 
              n_replicates=300, 
              ATE_estimator_fn=ATE_estimator_fn_interaction,
              CV_estimator_fn=CV_estimator_fn_interaction,
             )

In [33]:
sample_cov = np.cov(np.array([ATE_hat_samples, CV_samples]), ddof=1)

# Get optimal control variates coefficient
cov_ATE_CV = sample_cov[0][1]
var_CV = sample_cov[1][1]
optimal_CV_coeff = cov_ATE_CV / var_CV
print("optimal CV coefficient:", optimal_CV_coeff)

optimal CV coefficient: 0.0006768412149914828


In [34]:
# Get variance/bias of ATE estimators with and without CV.
CV_samples, ATE_hat_samples, ATE_hat_CV_samples = bootstrap.run_bootstrap_df(
    df_obs=df_orig, 
    df_bias=df_selection_bias, 
    n_replicates=300, # Try increasing this
    ATE_estimator_fn=ATE_estimator_fn_interaction,
    CV_estimator_fn=CV_estimator_fn_interaction,
    optimal_CV_coeff=optimal_CV_coeff)

In [35]:
ATE_true = 0.106286795757474

ATE_var = np.var(np.array(ATE_hat_samples), ddof=1)
print(">>> Variance of ATE estimator:", ATE_var)

ATE_bias = np.mean(np.array(ATE_hat_samples)) - ATE_true
print(">>> Bias of ATE estimator:", ATE_bias)

ATE_CV_var = np.var(np.array(ATE_hat_CV_samples), ddof=1)
print(">>> Variance of ATE estimator with CV:", ATE_CV_var)

ATE_CV_bias = np.mean(np.array(ATE_hat_CV_samples)) - ATE_true
print(">>> Bias of ATE estimator with CV:", ATE_CV_bias)

>>> Variance of ATE estimator: 0.00045762030740850696
>>> Bias of ATE estimator: 0.0006785186885944761
>>> Variance of ATE estimator with CV: 0.0004415051754272691
>>> Bias of ATE estimator with CV: 0.0021320231888025676
